In [1]:
%load_ext autoreload
%autoreload 2

from perturbations import *
from representation import *
from ls import *
import jupyter_utils as utils
from ea import *
from cross import *

import numpy as np
import matplotlib.pyplot as plt
import tsplib95

In [111]:
# load problem from the library
problem = tsplib95.load("./archives/bayg29.tsp/bayg29.tsp")
#
# "./archives/brazil58.tsp/brazil58.tsp"
#  ./archives/bayg29.tsp/bayg29.tsp
n = problem.dimension
distance_matrix = utils.weights_to_matrix(problem.edge_weights)
print("size of matrix: ", n)
# best = 1610
fitness = fitness_wrapper(distance_matrix)

size of matrix:  29


In [99]:
# load problem from the library
problem = tsplib95.load("./archives/brazil58.tsp/brazil58.tsp")
#
# "./archives/brazil58.tsp/brazil58.tsp"
#  ./archives/bayg29.tsp/bayg29.tsp
n = problem.dimension
distance_matrix = utils.weights_to_matrix(problem.edge_weights)
print("size of matrix: ", n)
# best = 1610
fitness = fitness_wrapper(distance_matrix)

size of matrix:  58


In [101]:
# load problem from the library
problem = tsplib95.load("./archives/brg180.tsp/brg180.tsp")
#
# "./archives/brazil58.tsp/brazil58.tsp"
#  ./archives/bayg29.tsp/bayg29.tsp
n = problem.dimension
distance_matrix = utils.blocky_weights_to_sym_matrix(problem.edge_weights, n)
print("size of matrix: ", n)
fitness = fitness_wrapper(distance_matrix)

size of matrix:  180


In [96]:
perturbation = perturb_invert
#           perturb_move
#           perturb_invert
#           perturb_switch

ls = ls_first_improvement

# init_fn = better_init_wrapper(size=n, dist_matrix=distance_matrix)
init_fn = constructive_heuristics_init_wrapper(
    size=n, dist_matrix=distance_matrix
)  # here select init!
max_evals = 20_000
trials = 10


fitness_list = []


for trial in range(trials):

    results = ls(
        fitness_fn=fitness,
        initialisation_fn=init_fn,
        perturbation_fn=perturbation,
        stop_cond=max_evals,
    )

    fitness_list.append(results["best_fitness"])


fitness_list = np.array(fitness_list)


print(np.mean(fitness_list), np.std(fitness_list))
print(fitness_list)

4472.0 1708.132313376221
[2690. 5650. 2400. 6330. 2010. 5830. 2580. 6340. 5430. 5460.]


In [ ]:
perturbation = perturb_invert
#           perturb_move
#           perturb_invert
#           perturb_switch

ls = ls_first_improvement_with_reinit

init_fn = better_init_wrapper(size=n, dist_matrix=distance_matrix)

max_evals = 20_000
trials = 10

fitness_list = []
for trial in range(trials):

    results = ls(
        fitness_fn=fitness,
        initialisation_fn=init_fn,
        perturbation_fn=perturbation,
        stop_cond=max_evals,
    )

    fitness_list.append(results["best_fitness"])


fitness_list = np.array(fitness_list)

print(np.mean(fitness_list), np.std(fitness_list))
print(fitness_list)

In [103]:
perturbation = invert_subseq
#           switch_cities
#           move_cities
#           invert_subseq


ls = ls_best_improvement


init_fn = constructive_heuristics_init_wrapper(size=n, dist_matrix=distance_matrix)


max_evals = 20_000
trials = 10


fitness_list = []


for trial in range(trials):

    results = ls(
        fitness_fn=fitness,
        initialisation_fn=init_fn,
        perturbation_operation=perturbation,
        stop_cond=max_evals,
    )

    fitness_list.append(results["best_fitness"])


fitness_list = np.array(fitness_list)


print(np.mean(fitness_list), np.std(fitness_list))
print(fitness_list)

3349.0 1697.1001738259295
[5430. 5430. 1980. 1960. 1960. 1960. 1960. 5430. 1960. 5420.]


In [113]:
init = better_init_wrapper(size=n, dist_matrix=distance_matrix)
# will be reused inside the function to init the population

# select crossover!
crossover = crossover_scx
# crossover_pmx
# crossover_scx

perturb = perturb_invert

# population management
replacement = combined_replacement
selection = tournament_selection

# generations consts
prob_cross = 0.5
prob_mut = 0.1
gen_size = 1000
num_offsprings = 1500
tourn_size = 50
stoping_criterion = 20_000

# evaluation const
trials = 1

print(
    "approx number of offsprings created from each generation: ",
    num_offsprings * prob_cross,
)
last_history = []
for i in range(trials):
    results = ea_alg(
        fitness_fn=fitness,
        init_fn=init,
        selection_fn=selection,
        crossover_fn=crossover,
        mutation_fn=perturb,
        replacement_fn=replacement,
        generation_size=gen_size,
        distance_matrix=distance_matrix,
        p_cross=prob_cross,
        offspring_num=num_offsprings,
        p_mut=prob_mut,
        tournament_size=tourn_size,
        max_evaluations=stoping_criterion,
    )
    list_of_results.append(results["best_fitness"])

list_of_results = np.array(list_of_results)


print(np.mean(list_of_results), np.std(list_of_results))
print(list_of_results)

approx number of offsprings created from each generation:  750.0
starting evolution:  best fitness of init 4323.0
1642.0 0.0
[1642.]


### Porovnání inicializací.

In [82]:
# init_fn = constructive_heuristics_init_wrapper(size=n, dist_matrix=distance_matrix)
# init_fn = random_init_wrapper(size=n)
init_fn = better_init_wrapper(size=n, dist_matrix=distance_matrix)

trials = 50
list_results = []
for i in range(trials):
    list_results.append(fitness(init_fn()))
list_results = np.array(list_results)
print(np.mean(list_results), np.std(list_results))
print(list_results)

801193.8 17460.97109441511
[803610. 803390. 774460. 823830. 798950. 805400. 795400. 816060. 835460.
 789980. 804610. 803610. 813470. 765000. 798530. 801280. 822880. 813050.
 827360. 828880. 817170. 820020. 822230. 798460. 781070. 794050. 813490.
 790240. 810400. 764030. 776110. 804560. 803960. 798620. 794990. 810960.
 813380. 793510. 796050. 801540. 795800. 808710. 799990. 751530. 814600.
 805460. 778600. 769760. 789320. 815870.]
